In [9]:
import os
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Flatten,Dropout,Input,Activation,Dense
from tensorflow.keras.models import Sequential,Model,load_model
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import StratifiedKFold

In [3]:
df = pd.read_csv("/content/drive/My Drive/Study/dacon/mnist/test.csv",index_col=[0])
x_test = df.values[:,1:].reshape(-1,28,28,1).astype('float32')/255.0

In [4]:
datagen2 = ImageDataGenerator()
test_generator = datagen2.flow(x_test,shuffle=False)

In [ ]:
'/content/drive/My Drive/Study/dacon/mnist/models/02_03_image_generater_colab.h5'

'/content/drive/My Drive/Study/dacon/mnist/models/02_03_image_generater_colab.h5'

In [8]:
def make_acc_loss_csv(index):
    for i in range(3,index+1):
        model1 = load_model('/content/drive/My Drive/Study/dacon/mnist/AI_models/02_04_AI_val_loss_index_{}.h5'.format(i))
        model2 = load_model('/content/drive/My Drive/Study/dacon/mnist/AI_models/02_04_AI_val_accuracy_index_{}.h5'.format(i))
        df = pd.read_csv("/content/drive/My Drive/Study/dacon/mnist/test.csv",index_col=[0])
        x_test = df.values[:,1:].reshape(-1,28,28,1).astype('float32')/255.0
        datagen = ImageDataGenerator(height_shift_range=(-1,1),width_shift_range=(-1,1))
        test_generator = datagen.flow(x_test,shuffle=False)
        y_pred = model1.predict_generator(test_generator,verbose=True)
        y_pred = np.argmax(y_pred,axis=-1)
        y_pred2 = model2.predict_generator(test_generator,verbose=True)
        y_pred2 = np.argmax(y_pred2,axis=-1)
        df_sub = pd.read_csv('/content/drive/My Drive/Study/dacon/mnist/submission.csv',index_col=0)
        df_sub['digit'] = y_pred
        df_sub.to_csv('/content/drive/My Drive/Study/dacon/mnist/test_img_generator/val_loss_kfold_{}.csv'.format(i))
        df_sub['digit'] = y_pred2
        df_sub.to_csv('/content/drive/My Drive/Study/dacon/mnist/test_img_generator/val_accuracy_kfold_{}.csv'.format(i))
make_acc_loss_csv(16)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


640/640 [==============================] - 134s 209ms/step


In [ ]:
model=load_model('/content/drive/My Drive/Study/dacon/mnist/AI_models/02_04_AI_val_loss_index_13.h5')
y_pred = model.predict_generator(test_generator,verbose=True)
y_pred.shape

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


 30/640 [>.............................] - ETA: 5:10:37

KeyboardInterrupt: ignored

In [ ]:
def make_acc_loss_csv(index):
    for i in range(1,index+1):
        model1 = load_model('/content/drive/My Drive/Study/dacon/mnist/AI_models/02_04_AI_val_loss_index_{}.h5'.format(i))
        model2 = load_model('/content/drive/My Drive/Study/dacon/mnist/AI_models/02_04_AI_val_accuracy_index_{}.h5'.format(i))
        df = pd.read_csv("test.csv",index_col=[0])
        x_test = df.values[:,1:].reshape(-1,28,28).astype('float32')/255.0
        y_pred = model.predict(x_test)
        y_pred = np.argmax(y_pred,axis=-1)
        y_pred2 = model2.predict(x_test)
        y_pred2 = np.argmax(y_pred2,axis=-1)
        df_sub = pd.read_csv('submission.csv',index_col=0)
        df_sub['digit'] = y_pred
        df_sub.to_csv('./home_models/val_loss_kfold_{}.csv'.format(i))
        df_sub['digit'] = y_pred2
        df_sub.to_csv('./home_models/val_accuracy_kfold_{}.csv'.format(i))

In [15]:
def ensembles(index):
    accuracy=[]
    for i in index:
    #for i in range(1,index+1):
        a = pd.read_csv("/content/drive/My Drive/Study/dacon/mnist/AI_models/accuracy_kfold_{}.csv".format(i),index_col=0)
        accuracy.append(a.values)
    accuracy = np.concatenate(accuracy,axis=1)
            
    loss=[]
    for i in index:
    #for i in range(1,index+1):
        b = pd.read_csv("/content/drive/My Drive/Study/dacon/mnist/AI_models/loss_kfold_{}.csv".format(i),index_col=0)
        loss.append(b.values)        
    loss = np.concatenate(loss,axis=1)
    
    accuracy_list=[]
    for j in range(len(accuracy)):
        count_list=[0,0,0,0,0,0,0,0,0,0]
        k = list(accuracy[j])
        for n in k:
            count_list[n]+=1
        accuracy_mode = count_list.index(max(count_list))
        accuracy_list.append(accuracy_mode)
    accuracy_list = np.array(accuracy_list).reshape(-1,1)
    df = pd.read_csv("/content/drive/My Drive/Study/dacon/mnist/submission.csv",index_col=0)
    df['digit'] = accuracy_list
    df.to_csv('/content/drive/My Drive/Study/dacon/mnist/AI_models/accuracy_ensembles.csv')
    
    loss_list=[]
    for j in range(len(loss)):
        count_list=[0,0,0,0,0,0,0,0,0,0]
        k = list(loss[j])
        for n in k:
            count_list[n]+=1
        loss_mode = count_list.index(max(count_list))
        loss_list.append(loss_mode)
    loss_list = np.array(loss_list).reshape(-1,1)
    df = pd.read_csv("/content/drive/My Drive/Study/dacon/mnist/submission.csv",index_col=0)
    df['digit'] = accuracy_list
    df.to_csv('/content/drive/My Drive/Study/dacon/mnist/AI_models/loss_ensembles.csv')
    
    
        
    
    result = np.concatenate([accuracy,loss],axis=1)
    mode_list=[]
    for j in range(len(result)):
        count_list=[0,0,0,0,0,0,0,0,0,0]
        k = list(result[j])
        for n in k:
            count_list[n]+=1
        mode = count_list.index(max(count_list))
        mode_list.append(mode)
    mode_list = np.array(mode_list).reshape(-1,1)
    df = pd.read_csv("/content/drive/My Drive/Study/dacon/mnist/submission.csv",index_col=0)
    df['digit'] = mode_list
    df.to_csv('/content/drive/My Drive/Study/dacon/mnist/AI_models/both_ensembles'+'.csv')
    return 

In [16]:
ensembles([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16])